## Preamble

In [47]:
# noinspection PyUnresolvedReferences
import preamble

In [48]:
from os import getenv
from pathlib import Path

clinica_data_ci_dir = getenv("CLINICA_DATA_CI_DIR", str(Path.cwd().parent /  "clinica_data_ci"))

pipeline_dir = Path(clinica_data_ci_dir) / "data_ci" / "T1Linear"
pipeline_in_dir = pipeline_dir / "in"
pipeline_ref_dir = pipeline_dir / "ref"
pipeline_out_dir = pipeline_dir / "out"

## BIDS data grabber

Definition of the BDG interface.

In [49]:
from nipype.interfaces.io import BIDSDataGrabber

bids_data_grabber = BIDSDataGrabber(
    outfields=["T1w"],
    output_query={
        "T1w": {
            "datatype": "anat",
            "suffix": "T1w",
            "extension": [".nii.gz"],
        }
    }
)

In [50]:
from pathlib import PurePath
from pydra.mark import annotate, task


@task
@annotate({"return": {"dataset_description": dict}})
def read_dataset_description(path: PurePath):
    """Read the dataset description for a BIDS dataset.

    :param path: Path to the dataset.
    :return: The dataset description as a dictionary.
    """
    import json

    with open(path / "dataset_description.json") as f:
        return json.load(f)


@task
def write_derivative_description(dataset_description: dict, path: PurePath):
    """Write the dataset description for a derivative dataset.

    :param dataset_description: Dataset description of the BIDS dataset.
    :param path: Path to the derivative dataset.
    :return: None.
    """
    import json

    dataset_description["Name"] = f"T1 linear for {dataset_description['Name']}"
    dataset_description["DatasetType"] = "derivative"
    dataset_description["GeneratedBy"] = [{"Name": "Clinica", "Version": "0.7.0"}]

    with open(path / "dataset_description.json", mode="w") as f:
        json.dump(dataset_description, f, indent=4)

## Bias field correction

Definition of the BFC interface.

In [51]:
from nipype.interfaces.ants import N4BiasFieldCorrection

n4_bias_field_correction = N4BiasFieldCorrection(bspline_fitting_distance=300)

## Registration

Definition of the registration interface.

In [52]:
from nipype.interfaces.ants import RegistrationSynQuick

registration_syn_quick = RegistrationSynQuick(transform_type="a")

## Cropping

Definition of the cropping interface.

In [53]:
from pydra.mark import annotate, task

@task
@annotate({"return": {"cropped_image": str}})
def crop_image(input_image: str, template_image: str) -> str:
    from pathlib import Path
    from nilearn.image import resample_to_img

    cropped_image = Path.cwd() / Path(input_image).name.replace(".nii.gz", "_cropped.nii.gz")

    resample_to_img(
        source_img=str(input_image),
        target_img=str(template_image),
        force_resample=True
    ).to_filename(cropped_image)

    return cropped_image

## Template fetchers

In [54]:
from pathlib import PurePath

def download_file(url: str, to: str) -> PurePath:
    from shutil import copyfileobj
    from ssl import SSLContext
    from urllib.request import urlopen

    print(f"Downloading {url} to {to}...")

    response = urlopen(url=url, context=SSLContext())
    with open(to, mode="wb") as f:
        copyfileobj(response, f)

    return PurePath(to)


@task
@annotate({"return": {"mni_template_file": PurePath}})
def download_mni_template() -> PurePath:
    from pathlib import Path

    return download_file(
        url="https://aramislab.paris.inria.fr/files/data/img_t1_linear/mni_icbm152_t1_tal_nlin_sym_09c.nii.gz",
        to=str(Path.cwd() / "mni_icbm152_t1_tal_nlin_sym_09c.nii.gz"),
    )


@task
@annotate({"return": {"ref_template_file": PurePath}})
def download_ref_template() -> PurePath:
    from pathlib import Path

    return download_file(
        url="https://aramislab.paris.inria.fr/files/data/img_t1_linear/ref_cropped_template.nii.gz",
        to=str(Path.cwd() / "ref_cropped_template.nii.gz"),
    )

## Workflow constructors

In [55]:
from pydra import Workflow


def build_input_workflow(name: str = "input") -> Workflow:
    """Input workflow for the T1 linear pipeline.

    Responsible for fetching the MNI and reference templates and the T1w files
    of the input BIDS dataset.

    :param name: The name of the workflow.
    :return: The input workflow.
    """
    from pydra.tasks.nipype1.utils import Nipype1Task

    wf = Workflow(name=name, input_spec=["input_dir"])

    wf.add(download_mni_template(name="download_mni_template"))

    wf.add(download_ref_template(name="download_ref_template"))

    wf.add(
        Nipype1Task(
            name="bids_data_grabber",
            interface=bids_data_grabber,
            base_dir=wf.lzin.input_dir,
        )
    )

    wf.set_output([
        ("t1w_files", wf.bids_data_grabber.lzout.T1w),
        ("mni_template_file", wf.download_mni_template.lzout.mni_template_file),
        ("ref_template_file", wf.download_ref_template.lzout.ref_template_file),
    ])

    return wf


def build_core_workflow(name: str = "core") -> Workflow:
    """Core workflow for the T1 linear pipeline.

    :param name: The name of the workflow.
    :return: The core workflow.
    """
    from pydra.tasks.nipype1.utils import Nipype1Task

    wf = Workflow(name=name, input_spec=["t1w_file", "mni_template_file", "ref_template_file"])

    wf.add(
        Nipype1Task(
            name="n4_bias_field_correction",
            interface=n4_bias_field_correction,
            input_image=wf.lzin.t1w_file,
        )
    )

    wf.add(
        Nipype1Task(
            name="registration_syn_quick",
            interface=registration_syn_quick,
            fixed_image=wf.lzin.mni_template_file,
            moving_image=wf.n4_bias_field_correction.lzout.output_image,
        )
    )

    wf.add(
        crop_image(
            name="crop_image",
            interface=crop_image,
            input_image=wf.registration_syn_quick.lzout.warped_image,
            template_image=wf.lzin.ref_template_file,
        )
    )

    wf.set_output([
        ("t1w_corrected_file", wf.n4_bias_field_correction.lzout.output_image),
        ("t1w_registered_file", wf.registration_syn_quick.lzout.warped_image),
        ("t1w_cropped_file", wf.crop_image.lzout.cropped_image),
        ("xfm_file", wf.registration_syn_quick.lzout.out_matrix),
    ])

    return wf


def build_output_workflow(name: str = "output") -> Workflow:
    """Example of an output workflow.

    :param name: The name of the workflow.
    :return: The output workflow.
    """
    import pydra

    @pydra.mark.task
    @pydra.mark.annotate({"return": {"output_file": str}})
    def bids_writer_task(input_file, output_dir):
        """
        Task to write files to output_dir
        """
        import subprocess
        output_file = f"{output_dir}/{input_file}"
        print(f"{output_file}")

        return output_file

    wf = Workflow(name=name, input_spec=["input_file", "output_dir"])
    wf.add(bids_writer_task(name="bids_writer", input_file=wf.lzin.input_file, output_dir=wf.lzin.output_dir))
    wf.set_output([("output_file", wf.bids_writer.lzout.output_file)])

    return wf

## Workflow definition

In [56]:
workflow = Workflow(
    name="t1_linear",
    input_spec=["input_dir", "output_dir"],
    input_dir=pipeline_in_dir / "bids",
    output_dir=pipeline_out_dir,
)

wf_input = build_input_workflow()
wf_input.inputs.input_dir = workflow.lzin.input_dir
workflow.add(wf_input)

wf_core = build_core_workflow()
wf_core.inputs.t1w_file = wf_input.lzout.t1w_files
wf_core.inputs.mni_template_file = wf_input.lzout.mni_template_file
wf_core.inputs.ref_template_file = wf_input.lzout.ref_template_file
workflow.add(wf_core.split("t1w_file"))

wf_output = build_output_workflow()
wf_output.inputs.input_file = wf_core.lzout.t1w_cropped_file
wf_output.inputs.output_dir = "/Users/ghislain.vaillant"
workflow.add(wf_output)

dataset_description_reader_task = read_dataset_description(
    name="read_dataset_description",
    path=workflow.lzin.input_dir,
)
workflow.add(dataset_description_reader_task)

dataset_description_writer_task = write_derivative_description(
    name="write_dataset_description",
    path=workflow.lzin.output_dir,
    dataset_description=dataset_description_reader_task.lzout.dataset_description,
)
workflow.add(dataset_description_writer_task)

workflow.set_output([
    ("t1w_cropped_file", wf_output.lzout.output_file),
    ("xfm_file", wf_core.lzout.xfm_file),
])

## Workflow execution

In [ ]:
from pydra import Submitter

with Submitter(plugin="cf", n_procs=4) as submitter:
    submitter(workflow)

results = workflow.result(return_inputs=True)

print(results)

220601-16:45:21,261 nipype.workflow INFO:
	 [Node] Setting-up "bids_data_grabber" in "/private/var/folders/m3/r24ql8bj2h9_g1d970f9mr40000w73/T/tmpbmrtvl2s/Nipype1Task_b25e6a29623b8d959afe6a492cd74b7fa97000a74872aae9b09144c366d608e8/bids_data_grabber".
220601-16:45:21,264 nipype.workflow INFO:
	 [Node] Executing "bids_data_grabber" <nipype.interfaces.io.BIDSDataGrabber>
220601-16:45:21,658 nipype.workflow INFO:
	 [Node] Finished "bids_data_grabber", elapsed time 0.133579s.
